# Survey Bot - Phase 1  

### Questionnaire

In [8]:
import pandas as pd

# Read Excel file
qn_display = pd.read_excel("/Users/admin/ai-questionnaire-project/excel_qns.xlsx")  # Replace with your actual file path
# Fill NaN with empty strings
qn_display.fillna('', inplace=True)

# Display DataFrame
display(qn_display)


,Question_ID,Question_Type,Question_Text,Options_(if_applicable)
0,Q1,Fill-in-the-blank,How many days a week do you walk?,-
1,Q2,Multiple Choice,Do you drink tea?,"Yes,No"
2,Q3,Multiple Choice,How often do you eat fish?,"Daily,Weekly,Monthly,Never"
3,Q4,Fill-in-the-blank,How many days a week do you drink alcohol?,-
4,Q5,Multiple Choice,How many hours do you sleep?,"5 hrs,6 hrs,8 hrs,10 hrs"


### Kannada STT

##### Original text

In [ ]:
# I drink alcohol two days a week. I drink tea and eat fish three times a week, I sleep for 5 hours.
# /Users/admin/ai-questionnaire-project/audio_phase_1.mp3

##### STT Code

In [3]:
import tempfile
import torch
import os
import numpy as np
from scipy.io.wavfile import write
import logging
import speech_recognition as sr
from pydub import AudioSegment
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit import IndicProcessor
import threading
import sys
import time

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load IndicTrans model and tokenizer
MODEL_NAME = "ai4bharat/indictrans2-indic-en-1B"
config = BitsAndBytesConfig(load_in_8bit=True)
indic_en_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
indic_en_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, trust_remote_code=True)
ip = IndicProcessor(inference=True)

# Global variables to store output
stored_kannada_text = None
stored_english_translation = None

# ✅ Convert audio to WAV format using pydub
def convert_to_wav(audio_file):
    try:
        audio = AudioSegment.from_file(audio_file)
        temp_wav_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        audio.export(temp_wav_file.name, format="wav")
        return temp_wav_file.name
    except Exception as e:
        logging.error(f"Error converting file to WAV: {e}")
        return None

# ✅ Function to transcribe Kannada audio
def transcribe_audio(audio_file):
    try:
        logging.info(f"Converting audio file: {audio_file}")
        converted_file = convert_to_wav(audio_file)
        if not converted_file:
            return "Error converting audio file to WAV"

        recognizer = sr.Recognizer()
        with sr.AudioFile(converted_file) as source:
            audio = recognizer.record(source)

        logging.info("Running SpeechRecognition for transcription")
        transcription = recognizer.recognize_google(audio, language="kn-IN")
        logging.info(f"Transcription result: {transcription}")

        os.remove(converted_file)  # Remove temporary file after processing
        return transcription.strip()
    except sr.UnknownValueError:
        logging.error("SpeechRecognition could not understand the audio")
        return "Could not understand the audio"
    except sr.RequestError as e:
        logging.error(f"SpeechRecognition service error: {e}")
        return f"SpeechRecognition error: {e}"
    except Exception as e:
        logging.error(f"Error during transcription: {e}")
        return f"Error: {e}"

# ✅ Function to translate Kannada to English using IndicTrans
def translate_sentence(sentence):
    try:
        src_lang, tgt_lang = "kan_Knda", "eng_Latn"
        batch = [sentence]

        # Preprocess input
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize input
        inputs = indic_en_tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        )

        # Generate translation
        with torch.no_grad():
            generated_tokens = indic_en_model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode output
        with indic_en_tokenizer.as_target_tokenizer():
            generated_tokens = indic_en_tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Post-process result
        translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)
        return translations[0] if translations else "Translation failed."
    except Exception as e:
        logging.error(f"Error during translation: {e}")
        return f"Error: {e}"

def transcribe_audio_from_mic(language='kn-IN'):
    recognizer = sr.Recognizer()
    recognizer.dynamic_energy_threshold = True
    recognizer.pause_threshold = 1.0
    
    stop_event = threading.Event()

    with sr.Microphone() as source:
        print("Recording... Speak now!")
        recognizer.adjust_for_ambient_noise(source)

        audio_data = []
        start_time = time.time()  # Start time tracking

        def stop_recording():
            input("Press Enter to stop recording...\n")
            stop_event.set()

        stop_thread = threading.Thread(target=stop_recording)
        stop_thread.start()

        try:
            while not stop_event.is_set():
                audio = recognizer.listen(source)
                audio_data.append(audio)

            end_time = time.time()  # End time tracking
            duration = round(end_time - start_time, 2)  # Calculate duration in seconds

            print(f"Recording duration: {duration} seconds")

            if not audio_data:
                print("No audio captured.")
                return None, duration

            # Combine the audio chunks
            combined_audio = sr.AudioData(
                b''.join([a.get_raw_data() for a in audio_data]),
                source.SAMPLE_RATE,
                source.SAMPLE_WIDTH
            )

            # Attempt transcription
            try:
                text = recognizer.recognize_google(combined_audio, language=language)
                print("Transcription:", text)
                return text, duration
            except sr.UnknownValueError:
                print("Speech Recognition could not understand the audio.")
                return None, 0
            except sr.RequestError as e:
                print(f"Could not request results: {e}")
                return None, 0
        except Exception as e:
            print(f"Error: {e}")
            return None, 0
        finally:
            stop_event.set()
            stop_thread.join()

# ✅ Function to process input
def process_input(input_type, audio_file=None, text_input=None):
    if input_type == "1" and audio_file:
        kannada_text = transcribe_audio(audio_file)
    elif input_type == "2":
        transcription, duration = transcribe_audio_from_mic(language='kn-IN')

        if transcription:
            kannada_text = transcription
        else:
            return "Could not transcribe live audio", "No translation"
        
    elif input_type == "3" and text_input:
        kannada_text = text_input
    else:
        return "Invalid input", "Invalid input"
    
    # Ensure that kannada_text is a string (not a tuple)
    if isinstance(kannada_text, tuple):
        kannada_text = kannada_text[0]

    english_translation = translate_sentence(kannada_text)
    return kannada_text, english_translation

# ✅ Save output to file
def save_output_to_file(kannada_text, english_translation, file_path="output.txt"):
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(f"Kannada Text:\n{kannada_text}\n\n")
            f.write(f"English Translation:\n{english_translation}\n")
        logging.info(f"Output saved to {file_path}")
    except Exception as e:
        logging.error(f"Error saving output to file: {e}")

# ✅ Terminal-based input handling
def main():
    global stored_kannada_text, stored_english_translation
    
    print("\nSelect input type:")
    print("1. Audio File")
    print("2. Live Audio")
    print("3. Text Input")
    choice = input("Enter your choice (1/2/3): ").strip()

    if choice == "1":
        file_path = input("Enter path to the audio file: ").strip()
        if not os.path.exists(file_path):
            logging.error(f"Invalid file path: {file_path}")
            print("File not found. Please check the path and try again.")
            return
        kannada_text, english_translation = process_input("1", audio_file=file_path)

    elif choice == "2":
        logging.info("Starting live audio capture...")
        kannada_text, english_translation = process_input("2")

    elif choice == "3":
        text_input = input("Enter Kannada text: ").strip()
        kannada_text, english_translation = process_input("3", text_input=text_input)

    else:
        print("Invalid choice.")
        return

    # Store results globally for later use
    stored_kannada_text = kannada_text
    stored_english_translation = english_translation

    print("\n--- Results ---")
    print(f"Kannada Text: {kannada_text}")
    print(f"English Translation: {english_translation}")

    # Save output to file
    save_output_to_file(kannada_text, english_translation)

if __name__ == "__main__":
    main()

    # Display stored results for debugging or later use
    print("\n--- Stored Results ---")
    print(f"Stored Kannada Text: {stored_kannada_text}")
    print(f"Stored English Translation: {stored_english_translation}")



Select input type:
1. Audio File
2. Live Audio
3. Text Input


INFO:root:Converting audio file: /Users/admin/ai-questionnaire-project/flask_test_demo_audio.mp3
INFO:root:Running SpeechRecognition for transcription
INFO:root:Transcription result: ನಾನು ವಾರದಲ್ಲಿ ಎರಡು ದಿನ ಮದ್ಯಪಾನ ಮಾಡುತ್ತೇನೆ ನಾನು ಚಹಾ ಕುಡಿಯುತ್ತೇನೆ ಮತ್ತು ವಾರಕ್ಕೆ ಮೂರು ಬಾರಿ ಮೀನು ತಿನ್ನುತ್ತೇನೆ ನಾನು ಐದು ಗಂಟೆಗಳ ಕಾಲ ಮಲಗುತ್ತೇನೆ
/opt/anaconda3/envs/ai-questionnaire/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
INFO:root:Output saved to output.txt



--- Results ---
Kannada Text: ನಾನು ವಾರದಲ್ಲಿ ಎರಡು ದಿನ ಮದ್ಯಪಾನ ಮಾಡುತ್ತೇನೆ ನಾನು ಚಹಾ ಕುಡಿಯುತ್ತೇನೆ ಮತ್ತು ವಾರಕ್ಕೆ ಮೂರು ಬಾರಿ ಮೀನು ತಿನ್ನುತ್ತೇನೆ ನಾನು ಐದು ಗಂಟೆಗಳ ಕಾಲ ಮಲಗುತ್ತೇನೆ
English Translation: I drink alcohol two days a week, I drink tea and eat fish three times a week, I sleep for five hours.

--- Stored Results ---
Stored Kannada Text: ನಾನು ವಾರದಲ್ಲಿ ಎರಡು ದಿನ ಮದ್ಯಪಾನ ಮಾಡುತ್ತೇನೆ ನಾನು ಚಹಾ ಕುಡಿಯುತ್ತೇನೆ ಮತ್ತು ವಾರಕ್ಕೆ ಮೂರು ಬಾರಿ ಮೀನು ತಿನ್ನುತ್ತೇನೆ ನಾನು ಐದು ಗಂಟೆಗಳ ಕಾಲ ಮಲಗುತ್ತೇನೆ
Stored English Translation: I drink alcohol two days a week, I drink tea and eat fish three times a week, I sleep for five hours.


In [4]:
stored_english_translation

'I drink alcohol two days a week, I drink tea and eat fish three times a week, I sleep for five hours.'

### Accuracy

In [5]:
#accuracy (#gradio for full stt code of kannada- with speech recognition model for live audio)

from sentence_transformers import SentenceTransformer, util

# Load a multilingual model (supports Kannada and English)
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Original text
original_text = """I drink alcohol two days a week. I drink tea and eat fish three times a week, I sleep for 5 hours."""


#output from indictrans

translated_text = """"I drink alcohol two days a week, I drink tea and eat fish three times a week, I sleep for five hours."""


# Encode both sentences to embeddings
original_embedding = model.encode(original_text, convert_to_tensor=True)
translated_embedding = model.encode(translated_text, convert_to_tensor=True)

# Calculate semantic similarity
semantic_similarity = util.pytorch_cos_sim(original_embedding, translated_embedding).item()

# Keyword check (if needed)
keywords = ["no appetite", "vomit", "cold", "weak"]
flagged_keywords = [word for word in keywords if word in original_text.lower() and word not in translated_text.lower()]

# Results
print(f"Semantic Similarity: {semantic_similarity * 100:.2f}%")

if flagged_keywords:
    print(f"⚠️ Mismatched Keywords: {', '.join(flagged_keywords)}")
else:
    print("✅ No mismatched keywords detected.")


INFO:datasets:PyTorch version 2.6.0 available.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Semantic Similarity: 93.12%
✅ No mismatched keywords detected.


### QA (question answer) model

In [ ]:
import os
import json
import re
import pandas as pd
import spacy
from transformers import pipeline

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Function to extract the most relevant keyword (preferably a verb)
def extract_main_keyword(question):
    doc = nlp(question.lower())
    
    # Extract verbs first (action words)
    verbs = [token.text for token in doc if token.pos_ == "VERB"]
    
    # If no verbs found, fall back to nouns
    if verbs:
        return verbs[0]  # Return the first verb found
    else:
        nouns = [token.text for token in doc if token.pos_ == "NOUN"]
        return nouns[0] if nouns else None  # Return first noun if exists, else None

# Define file paths
INPUT_FILE_PATH = os.path.expanduser("/Users/admin/ai-questionnaire-project/excel_qns.xlsx")
OUTPUT_FILE_PATH = os.path.expanduser("/Users/admin/ai-questionnaire-project/completed_qns.xlsx")
VOCAB_FILE = "Final_vocab_dict.json"

# Load or create vocabulary
def load_or_create_vocab():
    if os.path.exists(VOCAB_FILE):
        with open(VOCAB_FILE, "r") as f:
            vocab = json.load(f)
    else:
        vocab = {
            "Options_yes_no": {
                "valid_options": ["Yes", "No"],  # List of valid Yes/No options
                "Yes": ["yes", "i do", "i drink", "i have"],
                "No": ["no", "i don't", "i do not", "never"]
            },
            "Options_frequency": {
                "valid_options": ["Daily", "Weekly", "Monthly", "Never"],  # List of valid frequency options
                "Daily": ["daily", "a day", "every day"],
                "Weekly": ["weekly", "a week", "every week"],
                "Monthly": ["monthly", "a month", "once a month"],
                "Never": ["never", "not at all"]
            },
            "num_mapping": {
                "one": "1", "two": "2", "three": "3", "four": "4", "five": "5",
                "six": "6", "seven": "7", "eight": "8", "nine": "9", "ten": "10",
                "once": "1", "twice": "2", "thrice": "3"
            }
        }

        with open(VOCAB_FILE, "w") as f:
            json.dump(vocab, f, indent=4)
    return vocab

vocab = load_or_create_vocab()

# Load Hugging Face QA model
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Load questionnaire from Excel
def load_questionnaire(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name="Questions")
        return df
    except Exception as e:
        print(f"Failed to load questionnaire. Error: {e}")
        return None

# Convert words to numbers using num_mapping
def convert_numbers(answer):
    words = answer.split()
    for i, word in enumerate(words):
        if word in vocab["num_mapping"]:
            words[i] = vocab["num_mapping"][word]
    return " ".join(words)

# Check if any exact key term from the question is in the transcription
def should_extract_answer(question, transcription):
    main_keyword = extract_main_keyword(question)
    if main_keyword:
        return main_keyword in transcription.lower()
    return False

# Extract answer using Hugging Face model
def extract_answer_with_model(question, transcription):
    try:
        result = qa_pipeline(question=question, context=transcription)
        return result['answer'].strip()
    except Exception as e:
        print(f"Model failed to extract answer: {e}")
        return None

# Process the predicted answer
def process_answer(predicted_answer, options):
    if not predicted_answer:
        return "No Answer Found"
    
    answer = convert_numbers(predicted_answer.lower().strip())

    if options == "-":
        for key in ["No", "Yes"]:
            if any(ans in answer for ans in vocab["Options_yes_no"][key]):
                return key
            
    if options == ["Yes","No"]:
        for key in ["No", "Yes"]:
            if any(ans in answer for ans in vocab["Options_yes_no"][key]):
                return key
    
    elif options == ["Daily","Weekly","Monthly","Never"]:
        for key, values in vocab["Options_frequency"].items():
            if any(ans in answer for ans in values):
                return key
    
    return answer


# Fill answers in the questionnaire
def fill_answers(df, transcription):
    for index, row in df.iterrows():
        question = row['Question_Text']
        question_type = row['Question_Type']
        options = row['Options_(if_applicable)']
        if options!='-':
            options=options.split(",")
        answer = "No Answer found"

        if question_type == "Multiple Choice":
            if options not in [vocab["Options_yes_no"]["valid_options"], vocab["Options_frequency"]["valid_options"]]:
                answer = "❗Engineer intervention required."
                df.at[index, 'Answer'] = answer
                continue  # Skip further processing and go to the next question
        
        if should_extract_answer(question, stored_english_translation):
            predicted_answer = extract_answer_with_model(question, stored_english_translation)
            print(question,predicted_answer,options)
            processed_answer = process_answer(predicted_answer, options)
            answer = processed_answer

        df.at[index, 'Answer'] = answer
    
    return df
print(stored_english_translation)
# Load questionnaire
df = load_questionnaire(INPUT_FILE_PATH)
if df is not None:
    df['Answer'] = "No Answer"
    updated_df = fill_answers(df, stored_english_translation)
    print("Updated questionnaire saved successfully!")
    display(df)


Device set to use mps:0


I drink alcohol two days a week, I drink tea and eat fish three times a week, I sleep for five hours.
Do you drink tea? I drink tea ['Yes', 'No']
How often do you eat fish? three times a week ['Daily', 'Weekly', 'Monthly', 'Never']
How many days a week do you drink alcohol? two -
Updated questionnaire saved successfully!


,Question_ID,Question_Type,Question_Text,Options_(if_applicable),Answer
0,Q1,Fill-in-the-blank,How many days a week do you walk?,-,No Answer found
1,Q2,Multiple Choice,Do you drink tea?,"Yes,No",Yes
2,Q3,Multiple Choice,How often do you eat fish?,"Daily,Weekly,Monthly,Never",Weekly
3,Q4,Fill-in-the-blank,How many days a week do you drink alcohol?,-,2
4,Q5,Multiple Choice,How many hours do you sleep?,"5 hrs,6 hrs,8 hrs,10 hrs",❗Engineer intervention required.
